# CHCO Redacted OR Percent Utilization Tool

## Imports

In [1]:
##################################
####### run this code first ######
##################################

#library imports
import pandas as pd
pd.set_option('display.max_rows', 250, 'display.max_columns', None) #set pd display options

import datetime as dt
from datetime import datetime, timedelta, date
import calendar

import numpy as np
import matplotlib.pyplot as plt #visualizations
from collections import OrderedDict #ordered dict lib

#suppress warning for depreciated fxn
import warnings
warnings.filterwarnings('once')

#maps folder for functions
import sys
sys.path.append(r'/redacted_folder_pathway_to_orutil')

#functions stored in this file
from orutil import orutil

### ▼ ▼ ▼ enter paths to the data here ▼ ▼ ▼

In [3]:
#surgeon and snapshot csv to be read in
#NOTE: make sure it's saved as .csv and NOT Excel file
surgn_csv = '..\orutil\data\surgeon_data.csv'
snapshot_csv = '..\orutil\data\snapshot_data.csv'

### ▲ ▲ ▲ enter paths to the data here ▲ ▲ ▲

***

<div class="alert alert-block alert-success">
<b>I: This section explores the data and creates lists to use in timeseries creation in section II. </b>
</div>

### Filter the surgeon csv, leaving only columns needed for analysis

In [4]:
filtSurgnDF = orutil.filterSurgnDF(surgn_csv) #reads in surgeon csv
filtSurgnDF.head() #view top of data frame

### View all the surgeons in df

In [ ]:
orutil.getAllSurgns(filtSurgnDF)

### Get all the dates each surgeon worked in the dataset

In [ ]:
#run above fxn with filtered dataset
surgnDict = orutil.makeSurgnDict(filtSurgnDF)
surgnDict

### Make dictionary of dates worked for all surgeon, dow combinations (values not specific to OR yet)

In [ ]:
dowDict = {0:'Monday',
           1:'Tuesday',
           2:'Wednesday',
           3:'Thursday',
           4:'Friday'}
dowList = list(dowDict.values())

In [ ]:
#call fxn to create dict with surgeon, dow (values do NOT reflect specific OR... yet)
unspecPopDict = orutil.make_unspec_pop_dict(filtSurgnDF, surgnDict, dowList)
unspecPopDict

### <font color = chocolate> This function corrects the values in unspecPopDict to reflect only ORs used on the specific dates (~ 30 sec runtime). </font>

In [ ]:
# correct unspecpopDict values so they reflect ORs as well
popDict = orutil.spec_pop_dict(filtSurgnDF, unspecPopDict) 
popDict

### Remove dates that exist in surgeon dataset but not in snapshot dataset

In [ ]:
#first pull in snapshot csv to compare
filtSnapDF = orutil.filterSnapDF(snapshot_csv)
filtSnapDF

In [ ]:
#remove dates that exist in surgn data but not in snapshot data
#this means procedure was ultimately canceled, or removed on that day
#it was in there because at one point it WAS scheduled on that day
        
popDict_no_empties = orutil.removeEmptyDFs(popDict, filtSnapDF, verbose=False)
popDict_no_empties

### Look at the whole population - tally the values (aka blocks, dates) in popDict

In [ ]:
#tallies the values (blocks) from popDict
talliedDict = orutil.tally_popDict_dates(popDict_no_empties)
talliedDict

### Make talliedDict a dataframe for easier viewing of whole population

In [ ]:
#make talliedDict a df
popdf = orutil.makeTalliedDictDF(talliedDict)
popdf

### Below cell will save the dataframe as a csv

In [ ]:
#uncomment and run this cell if you want to save as csv
# popdf.to_csv('dataset_population_tallied.csv')

<div class="alert alert-block alert-success">
<b>II: This section creates the trendline for given criteria </b>
</div>

<div class="alert alert-block alert-info">
<b>------------------------ ▼ ▼ ▼ enter parameters for trendline to see here ▼ ▼ ▼ -------------------------</b>
</div>

<div class="alert alert-block alert-info">
<b>------------------------ ▼ ▼ ▼ enter parameters for trendline to see here ▼ ▼ ▼ -------------------------</b>
</div>

In [4]:
#################################
### Enter filter info here #####
#################################

#surgeon csv you want to read in
surgn_csv = '..\orutil\data\surgeon_data.csv'
snapshot_csv = '..\orutil\data\snapshot_data.csv'

#manually enter primary surgeon list from above
# NOTE: these have been changed to reflect redactions,
# for the analysis, the project used actual surgeon names
all_surgns_dict = {0:'Surgeon B',
                   1:'Surgeon A',
                   2:'Surgeon C'}
owner = all_surgns_dict[1] # <========================= choose owner here
print(f'The primary surgeon is:', owner)

#day of week
dowDict = {0:'Monday',
           1:'Tuesday',
           2:'Wednesday',
           3:'Thursday',
           4:'Friday'}
dowList = list(dowDict.values())
dow = dowDict[0] # <=============================== choose DOW here
print(f'The day of week is:', dow)

#enter operating room number (e.g., 1111)
# NOTE: these have been changed to reflect redactions,
# for the analysis, the project used actual OR identifiers
ORnum = 1111 # <============================= choose OR number here
print(f'The OR is:', ORnum)

#number of lookback days (ie. 180, 90, 60)
num_lookback_days = 180 # <============================ enter number of lookback days here
print(f'The number of lookback days is:', num_lookback_days)

The primary surgeon is: Surgeon A
The day of week is: Monday
The OR is: 1111
The number of lookback days is: 180


<div class="alert alert-block alert-info">
<b>------------------------ ▲ ▲ ▲ enter parameters for trendline to see here ▲ ▲ ▲ -------------------------</b>
</div>

<div class="alert alert-block alert-info">
<b>------------------------ ▲ ▲ ▲ enter parameters for trendline to see here ▲ ▲ ▲ -------------------------</b>
</div>

# Find blocks - toggle granularity here

In [ ]:
#call fxn - toggle owner, dow, roomID here to go up/down in granularity
owner_dow_OR_dict = orutil.find_blocks(popDict_no_empties, owner, dow, ORnum)
owner_dow_OR_dict

### Add date to key, find denom, cases, and minutes to calc numer - store it all in a dict

In [ ]:
agg_dates_to_plot = orutil.agg_dates_to_plot(filtSurgnDF, filtSnapDF, owner_dow_OR_dict)
agg_dates_to_plot

### Count the aggregated days for reporting

In [ ]:
#count aggregated days for report
len(list(agg_dates_to_plot))

### Make ordered dict out of each key, val pair for plotting temporal buildup of cases

In [ ]:
plotDicts = orutil.agg_plot_dicts_per_date(filtSurgnDF, agg_dates_to_plot)
plotDicts

### Convert each date's temporal utilization buildup into a plottable key/value pairing of daysback and percent utilization and stores as a dictionary

In [ ]:
ts_dict = orutil.ts_plots(filtSurgnDF, agg_dates_to_plot, plotDicts, num_lookback_days, verbose=False)
ts_dict

### Combine each date into a key/value pair of days back and each % utilization value as a list for each daysback key

In [ ]:
xy_dict = orutil.get_xy_from_ts_dict(ts_dict, agg_dates_to_plot, verbose=False)
xy_dict

### Create two lists from the previous dictionary:
* a list of means to plot the central trendline of the model
* a list of one standard deviation to plot the confidence interval for each point in the model

In [ ]:
#calc predictive model from data - mean trendline, CI banding

mnList, ciList = orutil.calc_ts_model(xy_dict, verbose=False)
print(f'mnList:', mnList, '\n')
print(f'ciList:', ciList)

<div class="alert alert-block alert-success">
<b>III: This section plots the trendline for given criteria </b>
</div>

# PLOT RESULTS

In [ ]:
#set the figure size
plt.figure(figsize=(12, 12))

#set x and y values from above dictionary
x = list(xy_dict.keys())
y = mnList #central trendline from means of each block

fig, ax = plt.subplots()
fig.set_size_inches(12, 8)

# Plot the function
plt.plot(x, y)

#adjust y-axis window
x1,x2,y1,y2 = plt.axis()  
plt.axis((x1,x2,0,1.2))

#rest of plt attributes
plt.ylabel("% Utilization", fontsize=16, rotation=-90, labelpad=35)
ax.yaxis.set_label_position("right")
plt.xlabel("Days Back From Procedure Date", fontsize=16, labelpad=20)
plt.axhline(y=1, color='cadetblue', linestyle='--')
plt.axhline(y=0.8, color='orange', linestyle='--')
plt.xticks(rotation = 45)
# plt.title(f'% Utilization of OR vs Days Back\nfrom Procedure Dates on '+
#           dow+'s\n'+ ' for '+ owner+ ' in %i' % ORnum )

### The below cell graphs the same as the above graph but with confidence banding

In [ ]:
#add CI banding
# plt.figure(figsize=(12, 12))
plt.rcParams['ytick.right'] = plt.rcParams['ytick.labelright'] = True
plt.rcParams['ytick.left'] = plt.rcParams['ytick.labelleft'] = False

x_1 = x
y_1 = mnList
err_1 = ciList

fig, ax = plt.subplots()
fig.set_size_inches(12, 8)

plt.errorbar(x=x_1, y=y_1, yerr=err_1, color="gray", capsize=3,
             linestyle="None",
             marker="s", markersize=7, mfc="black", mec="black")


# #set y axis to always start at 0
x1,x2,y1,y2 = plt.axis()  
plt.axis((x1,x2,-0.1,1.2))

#rest of plt attributes
plt.ylabel("% Utilization", fontsize=16, rotation=-90, labelpad=35)
plt.xlabel("Days Back From Procedure Date", fontsize=16, labelpad=20)
plt.axhline(y=1, color='cadetblue', linestyle='--')
plt.axhline(y=0.8, color='orange', linestyle='--')
plt.xticks(rotation = 45)
ax.yaxis.set_label_position("right")
# plt.title(f'% Utilization of OR vs Days Back\nfrom Procedure Dates on '+
#           dow+'s\n'+ ' for '+ owner+ ' in %i' % roomID )
#abbreviated title for dows
# plt.title("$\\bf{Surgeon\ A\ -\ }$" + r"$\bf{" + str(dow) + "s" + "}$" +
#           "\n% Utilization of ORs vs Days Back from Procedure Dates", fontsize=16, pad=20)
#abbreviated title for Surgeon Only
plt.title("$\\bf{Surgeon\ C\ -\ Department\ III}$" +
          "\n% Utilization of ORs vs Days Back from Procedure Dates", fontsize=16, pad=20)
plt.show()